In [1]:
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy import sparse
from dataset import *
from dataloader import *
from loss import *
from model import *
from torch.nn import Softmax
import os
import bottleneck as bn
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
if torch.cuda.is_available():
        cuda_aval = True

device = torch.device("cuda" if cuda_aval else "cpu")
print("Now using device : ", device)
###################
print("Load and Preprocess Movielens dataset")
# Load Data
uim = pd.read_csv("/opt/ml/input/workspace/CF/DL CF/Mult-VAE/new_user_movie_interaction.csv")
#removed_uim = pd.read_csv("/opt/ml/input/workspace/Item_Removal/1000_removed.csv")
user_id = uim["user"].to_numpy()
uim_np = uim.to_numpy()[:, 1:] # 첫번째 열은 user_id이기 때문에 제외
print("Done!")

Now using device :  cuda
Load and Preprocess Movielens dataset
Done!


In [12]:
removed_uim = pd.read_csv("/opt/ml/input/workspace/Item_Removal/future_removed_uim.csv")
removed_uim_np = removed_uim.to_numpy()[:, 1:]

In [3]:
# 필요한 데이터 로드
bad_users_03 = pd.read_csv("/opt/ml/input/workspace/TestRecall/bad_user_id03.csv", header=None).to_numpy().reshape(-1)
bad_users_02 = pd.read_csv("/opt/ml/input/workspace/TestRecall/bad_user_id_02.csv", header=None).to_numpy().reshape(-1)
good_users_05 = pd.read_csv("/opt/ml/input/workspace/TestRecall/good_user_id05.csv", header=None).to_numpy().reshape(-1)
raw_data_og = pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")
raw_data = raw_data_og.groupby("user")["item"].apply(list) # 유저 별로 시청한 데이터를 리스트로 정리
#years = pd.read_csv("/opt/ml/input/data/train/years.tsv", delimiter = "\t")
#titles = pd.read_csv("/opt/ml/input/data/train/titles.tsv", delimiter="\t")
genre_og = pd.read_csv("/opt/ml/input/data/train/genres.tsv", delimiter="\t")
genre = genre_og.groupby("item")["genre"].apply(list) # 각 영화별 장르를 리스트로 정리
#views = pd.read_csv("views.csv")

In [9]:
genres_dict = dict()
for key in genre.keys() :
    gr = genre.loc[key]
    genres_dict[key] = gr

In [4]:
unique_sid = pd.read_csv("/opt/ml/input/data/train/pro_sg/unique_sid.txt", header=None)
item_id = unique_sid.to_numpy().reshape(6807)

In [8]:
unpop_gr = pd.read_csv("/opt/ml/input/workspace/User Item EDA/unpopular_genres_by_user.csv", header=None).to_numpy()[:,1:]

In [5]:
#load model
f = open("model_MVAE200_og.pt", 'rb')
model  = torch.load(f).to(device)

In [24]:
model.eval()
result = list()
for i in range(31360) : 
    user = user_id[i]
    unpop = unpop_gr[i]
    data_np = uim_np[i] # 유저 한 명의 상호작용 벡터 추출
    #removed_data = removed_uim_np[i]
    data = torch.FloatTensor(data_np) # 모델에 돌리기 위해 tensor로 변환
    data = data.reshape(1,-1).to(device) # 모델 안에서 

    probability, _, _ = model(data) # for MVAE
    #probability = model(data) # for DAE
    #probability = model(data, calculate_loss=False) # for RecVAE
    probability_np = np.array(probability.cpu().detach())[0]
    
    #probability_np[removed_data.nonzero()] = -np.inf
    probability_np[data_np.nonzero()] = -np.inf
    for j in np.where(data_np == 0)[0] :
        itid = item_id[j]
        for genre in genres_dict[itid] :
            if genre in unpop :
                probability_np[j] = -np.inf
                break
    idx = np.argpartition(probability_np, -10)[-10:]
    candidates = [[index, probability_np[index]] for index in idx]
    candidates.sort(key=lambda x:x[1], reverse = True)
    items = item_id[idx]
    for candidate in candidates :
        result.append((user, item_id[candidate[0]]))
    if i%5000 == 0 :
        print(f"{i}/31360 complete")

0/31360 complete
5000/31360 complete
10000/31360 complete
15000/31360 complete
20000/31360 complete
25000/31360 complete
30000/31360 complete


In [25]:
info = pd.DataFrame(result, columns=['user','item'])
info.to_csv("submission_MVAE200_BadGenreRemoved.csv",index=False)
print("Inference Done!")

Inference Done!
